In [ ]:
import pandas as pd

data = pd.read_csv('california_housing.csv')

data.dropna(inplace=True)

categorical_columns = [
    'Type', 'Heating', 'Cooling', 'Region', 'City', 'State', 'Flooring', 'Heating features',
    'Cooling features', 'Appliances included', 'Laundry features', 'Parking features'
]
data = pd.get_dummies(data, columns=categorical_columns, drop_first=True)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
numerical_columns = [
    'Year built', 'Lot', 'Bedrooms', 'Bathrooms', 'Full bathrooms',
    'Total interior livable area', 'Total spaces', 'Garage spaces',
    'Elementary School Score', 'Elementary School Distance', 'Middle School Score',
    'Middle School Distance', 'High School Score', 'High School Distance',
    'Tax assessed value', 'Annual tax amount'
]
data[numerical_columns] = scaler.fit_transform(data[numerical_columns])

In [ ]:
import pymc as pm

with pm.Model() as model:
    alpha = pm.Normal('alpha', mu=0, sigma=10)
    betas = pm.Normal('betas', mu=0, sigma=10, shape=len(data.columns) - 1)
    
    mu = alpha + pm.math.dot(data.drop('Listed Price', axis=1).values, betas)
    
    sigma = pm.HalfNormal('sigma', sigma=1)
    Y_obs = pm.Normal('Y_obs', mu=mu, sigma=sigma, observed=data['Listed Price'])

    trace = pm.sample(500, return_inferencedata=False)

In [ ]:
import arviz as az

az.plot_trace(trace)